#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo

In [2]:
from pymongo import MongoClient
client = MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


In [45]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [46]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)

l_d =df_ks.to_dict(orient='records')

### Importer les données

In [57]:
collection.insert_many(l_d)

Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

## Question 1  

In [5]:
cursor = (collection.find().sort([("usd pledge", 1)])).limit(5)
list(cursor)

[{'_id': 156812982,
  'name': 'SketchPlanner: Create and Plan- all in one beautiful book!',
  'category': 'Art Books',
  'main_category': 'Publishing',
  'currency': 'USD',
  'deadline': '2017-09-27',
  'goal': 13000.0,
  'launched': datetime.datetime(2017, 8, 28, 15, 47, 2),
  'pledged': 16298.0,
  'state': 'successful',
  'backers': 367,
  'country': 'US',
  'usd pledged': 2670.0,
  'usd_pledged_real': 16298.0},
 {'_id': 1835968190,
  'name': 'Proven sales with custom motorcycle accessories',
  'category': 'Sculpture',
  'main_category': 'Art',
  'currency': 'CAD',
  'deadline': '2016-02-24',
  'goal': 5000.0,
  'launched': datetime.datetime(2016, 1, 25, 17, 37, 10),
  'pledged': 1.0,
  'state': 'failed',
  'backers': 1,
  'country': 'CA',
  'usd pledged': 0.70814759,
  'usd_pledged_real': 0.7382253063635021},
 {'_id': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  '

## Question 2

In [40]:
cursor = collection.find({"state":"successful"}).count()
cursor

<ipython-input-40-31eeead3fbf2>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  cursor = collection.find({"state":"successful"}).count()


53040

## Question 3

In [107]:
cursor = collection.aggregate([{"$group" : {"_id" : "$category", "SumByCategory" : {"$sum" : 1}}}])
list(cursor)

[{'_id': 'Printing', 'SumByCategory': 83},
 {'_id': 'DIY', 'SumByCategory': 480},
 {'_id': 'Documentary', 'SumByCategory': 6498},
 {'_id': 'Digital Art', 'SumByCategory': 524},
 {'_id': 'Illustration', 'SumByCategory': 1263},
 {'_id': 'Technology', 'SumByCategory': 2690},
 {'_id': 'Performance Art', 'SumByCategory': 855},
 {'_id': 'Jazz', 'SumByCategory': 733},
 {'_id': 'Cookbooks', 'SumByCategory': 217},
 {'_id': 'Photo', 'SumByCategory': 58},
 {'_id': 'Fabrication Tools', 'SumByCategory': 96},
 {'_id': 'Glass', 'SumByCategory': 51},
 {'_id': 'Movie Theaters', 'SumByCategory': 90},
 {'_id': 'Interactive Design', 'SumByCategory': 146},
 {'_id': 'Academic', 'SumByCategory': 367},
 {'_id': 'Crafts', 'SumByCategory': 1834},
 {'_id': 'Fine Art', 'SumByCategory': 291},
 {'_id': 'Space Exploration', 'SumByCategory': 137},
 {'_id': 'Robots', 'SumByCategory': 237},
 {'_id': 'Calendars', 'SumByCategory': 112},
 {'_id': 'Gadgets', 'SumByCategory': 1210},
 {'_id': 'Hip-Hop', 'SumByCategory': 1555

## Question 4

In [126]:
import datetime
date_ = datetime.datetime.strptime("2016-01-01",'%Y-%m-%d')
cursor = collection.count_documents({"$and":[{"launched":{"$lt": date_}}, {"country":"FR"}]})
print(cursor)

330


## Question 5

In [83]:
cursor = collection.find({"country":"US"}, {"usd_pledge_real":{"gte":200000}})
list(cursor)

[{'_id': 872782264},
 {'_id': 1326492673},
 {'_id': 1688410639},
 {'_id': 156812982},
 {'_id': 1771789139},
 {'_id': 160394496},
 {'_id': 1248488072},
 {'_id': 424612381},
 {'_id': 1182908609},
 {'_id': 396454289},
 {'_id': 1522195027},
 {'_id': 1429288669},
 {'_id': 1441999253},
 {'_id': 1202442815},
 {'_id': 2116917173},
 {'_id': 2117684680},
 {'_id': 350628546},
 {'_id': 1849047042},
 {'_id': 295864582},
 {'_id': 1390262683},
 {'_id': 1041882251},
 {'_id': 1130940376},
 {'_id': 186326896},
 {'_id': 1025722498},
 {'_id': 1493170888},
 {'_id': 1007463174},
 {'_id': 2062002375},
 {'_id': 1035882318},
 {'_id': 1691723287},
 {'_id': 680665134},
 {'_id': 1864901261},
 {'_id': 519449839},
 {'_id': 1875047349},
 {'_id': 217543389},
 {'_id': 1263082934},
 {'_id': 764437823},
 {'_id': 1341393677},
 {'_id': 1149469807},
 {'_id': 1968352897},
 {'_id': 397524601},
 {'_id': 520797301},
 {'_id': 1195741430},
 {'_id': 2024072867},
 {'_id': 223758456},
 {'_id': 1594750207},
 {'_id': 726485587},
 {'_

## Question 6 

In [127]:
collection.create_index([("name", "text")])

OperationFailure: Index: { v: 2, key: { _fts: "text", _ftsx: 1 }, name: "name_text", weights: { name: 1 }, default_language: "english", language_override: "language", textIndexVersion: 3 } already exists with different options: { v: 2, key: { _fts: "text", _ftsx: 1 }, name: "description_text", weights: { description: 1 }, default_language: "english", language_override: "language", textIndexVersion: 3 }, full error: {'ok': 0.0, 'errmsg': 'Index: { v: 2, key: { _fts: "text", _ftsx: 1 }, name: "name_text", weights: { name: 1 }, default_language: "english", language_override: "language", textIndexVersion: 3 } already exists with different options: { v: 2, key: { _fts: "text", _ftsx: 1 }, name: "description_text", weights: { description: 1 }, default_language: "english", language_override: "language", textIndexVersion: 3 }', 'code': 85, 'codeName': 'IndexOptionsConflict'}

In [129]:
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'description_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('description', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [3]:
cursor = collection.find( { "$text": { "$search": "sport" } } )
next(cursor)

StopIteration: 